# Описание проекта

В данном проекте требовалось с помощью техники *Bootstrap* проанализировать возможные прибыль и риски для нефтедобывающей компании. На основе данных о трех регионах по 10 000 месторождений в каждом, об измерении качества нефти и объема ее запасов, нужно принять решение и бурении новых скважин.

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression

geo0 = pd.read_csv('/datasets/geo_data_0.csv')
geo1 = pd.read_csv('/datasets/geo_data_1.csv')
geo2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
for data in geo0, geo1, geo2:
    info = data.info()
    print(info)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory 

In [4]:
for data in geo0, geo1, geo2:
    head = data.head(5)
    print(head)

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [5]:
for data in geo0, geo1, geo2:
    duplic = data['id'].duplicated().sum()
    print(duplic)

10
4
4


In [6]:
for data in geo0, geo1, geo2:
    val = data['id'].value_counts()
    print(val)

fiKDv    2
74z30    2
bxg6G    2
HZww2    2
TtcGQ    2
        ..
auJQa    1
huQZX    1
ekvZk    1
P7qqt    1
OKZFU    1
Name: id, Length: 99990, dtype: int64
LHZR0    2
wt4Uk    2
5ltQ6    2
bfPNe    2
lEMOy    1
        ..
5j9x1    1
txYMo    1
8LC9c    1
VpfgX    1
Le7BU    1
Name: id, Length: 99996, dtype: int64
Vcm5J    2
KUPhW    2
xCHr8    2
VF7Jo    2
NXVa3    1
        ..
fhI1u    1
F599u    1
HbnRb    1
jHrKU    1
Ssaqr    1
Name: id, Length: 99996, dtype: int64


In [7]:
for data in geo0, geo1, geo2:
    drop = data['id'].drop_duplicates()
    print(drop)

0        txEyH
1        2acmU
2        409Wp
3        iJLyR
4        Xdl7t
         ...  
99995    DLsed
99996    QKivN
99997    3rnvd
99998    7kl59
99999    1CWhH
Name: id, Length: 99990, dtype: object
0        kBEdx
1        62mP7
2        vyE1P
3        KcrkZ
4        AHL4O
         ...  
99995    QywKC
99996    ptvty
99997    09gWa
99998    rqwUm
99999    relB0
Name: id, Length: 99996, dtype: object
0        fwXo0
1        WJtFt
2        ovLUW
3        q6cA6
4        WPMUX
         ...  
99995    4GxBu
99996    YKFjq
99997    tKPY3
99998    nmxp2
99999    V9kWn
Name: id, Length: 99996, dtype: object


В столбце id обнаружились одинаковые названия месторождений. Решаем удалить их, так как их довольно мало дял того, чтобы они играли сущесьвенную роль в работе модели.

# 2. Обучение и проверка модели

Для начала нужно разделить исходные данные на 2 выборки: обучающую и тестовую в соотношении 75 и 25%. И затем обучить можедь линейной регрессии на обучающей выборке и выполнить предсказания на валидационных данных по очереди для каждого региона.

Рассмотрим регион geo0:

In [8]:
train0, valid0 = train_test_split(geo0, test_size=0.25, random_state=12345)
features_train0 = train0.drop(['product', 'id'], axis=1)
target_train0 = train0['product']

features_valid0 = valid0.drop(['product', 'id'], axis=1)
target_valid0 = valid0['product'] 

print(features_train0.shape)
print(features_valid0.shape)

(75000, 3)
(25000, 3)


In [9]:
model = LinearRegression()
model.fit(features_train0, target_train0)
predicted_valid0 = model.predict(features_valid0)
mse = mean_squared_error(target_valid0, predicted_valid0)


print("Linear Regression")
print("MSE =", mse)
print("RMSE =", mse ** 0.5)
print("Mean =", predicted_valid0.mean())


Linear Regression
MSE = 1412.2129364399243
RMSE = 37.5794217150813
Mean = 92.59256778438038


Рассмотрим регион geo1:

In [10]:
train1, valid1 = train_test_split(geo1, test_size=0.25, random_state=12345)
features_train1 = train1.drop(['product', 'id'], axis=1)
target_train1 = train1['product']

features_valid1 = valid1.drop(['product', 'id'], axis=1)
target_valid1 = valid1['product'] 

print(features_train1.shape)
print(features_valid1.shape)

(75000, 3)
(25000, 3)


In [11]:
model = LinearRegression()
model.fit(features_train1, target_train1)
predicted_valid1 = model.predict(features_valid1)
mse = mean_squared_error(target_valid1, predicted_valid1)


print("Linear Regression")
print("MSE =", mse)
print("RMSE =", mse ** 0.5)
print("Mean =", predicted_valid1.mean())


Linear Regression
MSE = 0.7976263360391139
RMSE = 0.893099286775616
Mean = 68.728546895446


Рассмотрим регион geo2:

In [12]:
train2, valid2 = train_test_split(geo2, test_size=0.25, random_state=12345)
features_train2 = train2.drop(['product', 'id'], axis=1)
target_train2 = train2['product']

features_valid2 = valid2.drop(['product', 'id'], axis=1)
target_valid2 = valid2['product'] 

print(features_train2.shape)
print(features_valid2.shape)

(75000, 3)
(25000, 3)


In [13]:
model = LinearRegression()
model.fit(features_train2, target_train2)
predicted_valid2 = model.predict(features_valid2)
mse = mean_squared_error(target_valid2, predicted_valid2)


print("Linear Regression")
print("MSE =", mse)
print("RMSE =", mse ** 0.5)
print("Mean =", predicted_valid2.mean())

Linear Regression
MSE = 1602.3775813236196
RMSE = 40.02970873393434
Mean = 94.96504596800489


Данные поделили верно, видно из соотношения датасетов.

В результате видим, что самые большие средние запасы сырья находятся в регионе 3. При этом значение rmse довольно высокое.
Большое значение rmse региона говорит о большом разбросе ошибок, эти значения самые высокие для geo 0 и 2. 


# 3. Подготовка к расчёту прибыли

Запишем условия задачи в глобальные переменные.

all_points - сколько всего месторождений 

total_budget - бюджет всего региона

budget_of_one - бюджет одного месторождения

barell_profit -  цена бареля нефти

cost_of_one - доход с каждой единицы продукта 

risk_treshold - минимальная вероятность убытков

In [14]:
all_points = 500
bootstrap_samples = 1000
total_budget = 10000000000
budget_of_one = 50000000
cost_of_one = 450000
barell_profit = 450
risk_treshold = 0.025

#посчитаем сколько баррелей нефти нужно разработать с одного месторождения, чтобы окупить вложения в разработку месторождения
# при условии, что обрабатываем 200 скважин
total_barell = budget_of_one / (barell_profit * 1000)
total_barell



111.11111111111111

In [15]:
print('Средний объём сырья в 200 скважинах в регионе geo0:',predicted_valid0.mean()* budget_of_one)
print('Средний объём сырья в 200 скважинах в регионе geo1:',predicted_valid1.mean()* budget_of_one)
print('Средний объём сырья в 200 скважинах в регионе geo2:',predicted_valid2.mean()* budget_of_one)

Средний объём сырья в 200 скважинах в регионе geo0: 4629628389.219019
Средний объём сырья в 200 скважинах в регионе geo1: 3436427344.7723002
Средний объём сырья в 200 скважинах в регионе geo2: 4748252298.400245


Нужно 111 тыc баррелей на разработку одной сквжины, чтобы окупить все. Получается, что все регионы убыточны.



# 4. Расчёт прибыли и рисков 

Напишем функцию для расчета прибыли:

In [17]:
probabilities = predicted_valid2
target = target_valid2

def revenue(target, probabilities, count):
    top = probabilities.sort_values(ascending=False).head(count)
    total_profit = (target.loc[top.index][:count] * 1000 * barell_profit).sum()
    all_price = cost_of_one * count
    return total_profit - all_price#all_price


1. у меня немного дркгоая функция 
2. сначала она сортирует те самые 200 значений (или любое другое кол-во, котрое ты ей впишешь)значения, умножаем на 1000 и на цену бареля и суммируем 
3. потом находим значение дохода для скважин: умножаешь цену единцу продукта(450000) на кол-во отобранных скважин(200шт)
4. и потом ищещь весь профит :  значение из пункта 2 минус стоисмость дохода со скважин(пункт 3)

Бутстреп для каждого региона по очереди:
Вычислим среднюю выручку по регионам и 95% доверительный интервал:

In [18]:
state = np.random.RandomState(12345) 

values0 = []
target_valid0 = target_valid0.reset_index(drop=True)
for i in range(1000):
    target_subsample0 = target_valid0.sample(n=500, replace=True, random_state=state)
    valid_subsample0 = pd.Series(predicted_valid0)[target_subsample0.index]
    values0.append(revenue(target_subsample0, valid_subsample0, 200))

values0 = pd.Series(values0)
lower0 = values0.quantile(0.05)

upper0 = values0.quantile(0.95)
mean0 = values0.mean()
print("Geo0")
print("Средняя прибыль:", mean0)
print("95%-ый доверительный интервал:", lower0, ':', upper0)

risk0 = ((values0 < 0).mean()* 100, "%")
print("Риск убытков:", risk0)

Geo0
Средняя прибыль: 10335938526.910593
95%-ый доверительный интервал: 9878196885.653885 : 10791341454.360281
Риск убытков: (0.0, '%')


In [19]:
values1 = []
target_valid1 = target_valid1.reset_index(drop=True)
for i in range(1000):
    target_subsample1 = target_valid1.sample(n=500, replace=True, random_state=state)
    valid_subsample1 = pd.Series(predicted_valid1)[target_subsample1.index]
    values1.append(revenue(target_subsample1, valid_subsample1, 200))

values1 = pd.Series(values1)
mean1 = values1.mean()

lower1 = values1.quantile(0.05)
upper1 = values1.quantile(0.95)
mean1 = values1.mean()
print("Geo1")
print("Средняя прибыль:", mean1)
print("95%-ый доверительный интервал:", lower1, ':', upper1)

risk1 = ((values1 < 0).mean()* 100, "%")
print("Риск убытков:", risk1)

Geo1
Средняя прибыль: 10428259493.697329
95%-ый доверительный интервал: 10096591074.593811 : 10798640591.354553
Риск убытков: (0.0, '%')


In [46]:
values2 = []
target_valid2 = target_valid2.reset_index(drop=True)
for i in range(1000):
    target_subsample2 = target_valid2.sample(n=500, replace=True, random_state=state)
    valid_subsample2 = pd.Series(predicted_valid2)[target_subsample2.index]
    values2.append(revenue(target_subsample2, valid_subsample2, 200))

values2 = pd.Series(values2)
mean2 = values2.mean()

lower2 = values2.quantile(0.05)
upper2 = values2.quantile(0.95)
mean2 = values2.mean()
print("Geo2")
print("Средняя прибыль:", mean2)
print("95%-ый доверительный интервал:", lower2, ':', upper2)

risk2 = ((values2 > 10000000000).mean()* 100, "%")
print("Риск убытков:", risk2)
print()
risk3 = ((values2 < 0).mean()* 100, "%")
print("Риск убытков:", risk3)

Geo2
Средняя прибыль: 424212488.0991597
95%-ый доверительный интервал: 1291918.6677036325 : 860660712.8075709
Риск убытков: (0.0, '%')

Риск убытков: (4.9, '%')


Наибольшая средняя выручка у нас в geo2, но с вероятностью 95% мы получим чуть-чуть меньше прибыли. Похожая ситуация и с geo0. D geo1 разница между средней выручкой и 5-м квантилем  меньше, чем у остальных регионов. Похое модель тут и лучше. . Все три региона окупаются, но нужно оценить убытки.

Оценим убыточнсть наших регионов в процентах:

Для этого создадим таблицу с доходностью из списка из бутстрепа, потом отсортируем те, которые приносят менее 10 млрд рублей, ибо те, что меньше, просто окупаются и все и посчитаем их кол-во. Затем вычитаем их кол-во из общего кол-ва и делим на общее и выражаем в процентах. Для всех трех регионов выполняем это все.

In [181]:
values0pd = pd.DataFrame(values0)
values0pd.columns = ['revenue0']
values0pd

,revenue0
0,1.049509e+10
1,1.044639e+10
2,1.012108e+10
3,1.017528e+10
4,1.018199e+10
...,...
995,1.038410e+10
996,1.076600e+10
997,1.069599e+10
998,1.026422e+10


In [184]:
count_0 = values0pd.query('revenue0 > 10e+09').count()
percent_loss0 = ((1000-count_0) / 1000) * 100
percent_loss0


revenue0    11.0
dtype: float64

In [167]:
values1pd = pd.DataFrame(values1)
values1pd.columns = ['revenue1']
values1pd

,revenue1
0,1.030210e+10
1,1.003812e+10
2,1.003955e+10
3,1.042059e+10
4,1.046121e+10
...,...
995,1.049050e+10
996,1.043975e+10
997,1.023219e+10
998,1.097771e+10


In [185]:
count_1 = values1pd.query('revenue1 > 10e+09').count()
percent_loss01 = ((1000-count_1) / 1000) * 100
percent_loss01


revenue1    1.7
dtype: float64

In [169]:
values2pd = pd.DataFrame(values2)
values2pd.columns = ['revenue2']
values2pd

,revenue2
0,1.065534e+10
1,1.048367e+10
2,1.015293e+10
3,1.032539e+10
4,1.045596e+10
...,...
995,1.033749e+10
996,1.008146e+10
997,1.024362e+10
998,1.067518e+10


In [170]:
count_2 = values2pd.query('revenue2 > 10e+09').count()
percent_loss02 = ((1000-count_2) / 1000) * 100
percent_loss02

revenue2    11.6
dtype: float64

В итоге регион geo1 получился самым безубыточным, процент убыточности 1.7, это менее порога в задании в 2.5, его и возьмем в разработку!

В данном исследовании было дано 3 датасета с данными о трех регионах с месторождениями нефти. Мы провели предварительный анализ исходных данных, провели обработку, затем разделили датасет на 2 выборки, обучающую и валидационную 75 и 25 %. затем обучили модель линейной регресии на обуч-их данных и сделали предсказания на валид-х. Далее вычислили для каждого реиона среднюю выручку и 95% доверительный интервал. В результате выяснили, что самая высокая средняя выручка у реиона geo1. На последнем этапе посчитали процент убыточности каждого региона по отдельности, выяснили , что регион geo1 является наименеем убыточным, там процент убыточности 0.7, при условии задачи не более 2.5%. В итоге выбираем для разработки регион geo1